# Joint 3D analysis
This tutorial shows how to run a joint 3D map-based analysis using three example observations of the Galactic center region with CTA.

In [1]:
## Setup

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt

In [3]:
import os
from pathlib import Path
import numpy as np
import astropy.units as u
from astropy.coordinates import SkyCoord
from gammapy.data import DataStore
from gammapy.irf import EnergyDispersion, make_psf
from gammapy.maps import WcsGeom, MapAxis, Map, WcsNDMap
from gammapy.cube import MapMaker, MapEvaluator, PSFKernel, MapDataset
from gammapy.cube.models import SkyModel, SkyDiffuseCube, BackgroundModel
from gammapy.spectrum.models import PowerLaw, ExponentialCutoffPowerLaw
from gammapy.image.models import SkyGaussian, SkyPointSource
from gammapy.utils.fitting import Fit
from regions import CircleSkyRegion

## Prepare modeling input data

### Prepare input maps

We first use the `DataStore` object to access the CTA observations and retrieve a list of observations by passing the observations IDs to the `.get_observations()` method:

In [4]:
# Define which data to use and print some information
data_store = DataStore.from_dir("$GAMMAPY_DATA/cta-1dc/index/gps/")
data_store.info()


Data store:
HDU index table:
BASE_DIR: /home/ljouvin/installation_gammapy/gammapy-datasets/cta-1dc/index/gps
Rows: 24
OBS_ID: 110380 -- 111630
HDU_TYPE: ['aeff', 'bkg', 'edisp', 'events', 'gti', 'psf']
HDU_CLASS: ['aeff_2d', 'bkg_3d', 'edisp_2d', 'events', 'gti', 'psf_3gauss']

Observation table:
Observatory name: 'N/A'
Number of observations: 4


In [5]:
# Select some observations from these dataset by hand
obs_ids = [110380, 111140, 111159, 111630]
observations = data_store.get_observations(obs_ids)

In [6]:
energy_axis = MapAxis.from_edges(
    np.logspace(-1.0, 1.0, 10), unit="TeV", name="energy", interp="log"
)
geom = WcsGeom.create(
    skydir=(0, 0),
    binsz=0.02,
    width=(5, 4),
    coordsys="GAL",
    proj="CAR",
    axes=[energy_axis],
)

The `MapMaker` object is initialized with this reference geometry and a field of view cut of 4 deg.

The maps are prepared by calling the `.run()` method and passing the `observations`. The `.run()` method returns a Python `dict` containing a `counts`, `background` and `exposure` map. For the joint analysis, we compute the cube observations by observations run by run.

In [7]:
%%time
maker = MapMaker(geom, offset_max=4.0 * u.deg)
maps_list = {}

for obs in observations:
    maps_list[obs.obs_id]= maker.run([obs])

CPU times: user 4.18 s, sys: 328 ms, total: 4.51 s
Wall time: 4.51 s


### Prepare IRFs
PSF and Edisp are estimated for each observation at a specific source position src_pos
  

In [8]:
# for the PSF and edisp
src_pos = SkyCoord(0, 0, unit="deg", frame="galactic")
# define energy grid for edisp
energy = energy_axis.edges * energy_axis.unit

irf_list = {}
for obs in observations:
    irf_list[obs.obs_id]={}
    table_psf = make_psf(obs, src_pos)
    irf_list[obs.obs_id]["psf"] = PSFKernel.from_table_psf(table_psf, geom, max_radius="0.3 deg")

    offset=src_pos.separation(obs.pointing_radec)
    irf_list[obs.obs_id]["edisp"] = obs.edisp.to_energy_dispersion(offset, e_true=energy, e_reco=energy)

/home/ljouvin/installation_gammapy/gammapy/gammapy/cube/psf_kernel.py:110: RuntimeWarning: invalid value encountered in true_divide
  img += vals.value / vals.sum().value


In [9]:
#Save the maps for each observations
for obs_id in maps_list:
    path = Path("joint_analysis_cube") / "obs_{}".format(obs_id)
    path.mkdir(parents=True,exist_ok=True)
    for key in ["counts", "exposure", "background"]:
        filename="{}.fits.gz".format(key)
        maps_list[obs_id][key].write(path / filename, overwrite=True)
    for key in ["psf", "edisp"]:
        filename="{}.fits.gz".format(key)
        irf_list[obs_id][key].write(path / filename, overwrite=True)

## Likelihood fit

### Reading maps and IRFs
As first step we read in the maps and IRFs fir each observations

In [10]:
maps_dataset = {}
for obs in observations:
    path = Path("joint_analysis_cube") / "obs_{}".format(obs.obs_id)
    maps_dataset[obs.obs_id]= {}
    for keys in ["counts", "exposure", "background"]:
        maps_dataset[obs.obs_id][keys] = Map.read(path / "{}.fits.gz".format(keys))
    maps_dataset[obs.obs_id]["psf"] = PSFKernel.read(str(path / "psf.fits.gz"))
    maps_dataset[obs.obs_id]["edisp"] = EnergyDispersion.read(str(path / "edisp.fits.gz"))

In [11]:
#Define source model
spatial_model = SkyPointSource(lon_0="0.01 deg", lat_0="0.01 deg")
spectral_model = PowerLaw(
    index=2.2, amplitude="3e-12 cm-2 s-1 TeV-1", reference="1 TeV"
)
model = SkyModel(spatial_model=spatial_model, spectral_model=spectral_model)

In [15]:
# Create the dataset for each observation
dataset_list = []
for obs in observations:
    background_model = BackgroundModel(maps_dataset[obs.obs_id]["background"])
    dataset=MapDataset(
        model=model,
        counts=maps_dataset[obs.obs_id]["counts"],
        exposure=maps_dataset[obs.obs_id]["exposure"],
        psf=maps_dataset[obs.obs_id]["psf"],
        edisp=maps_dataset[obs.obs_id]["edisp"],
        background_model=background_model,
)
    dataset_list.append(dataset)
    


In [16]:
%%time
fit = Fit(dataset_list)
result = fit.run(optimize_opts={"print_level": 1})

FCN = 0.0 
 TOTAL NCALL = 44 
 NCALLS = 44 
 
 
 EDM = 0.0 
 GOAL EDM = 1e-05 
 
 UP = 1.0 
 
 
 
 
 Valid 
 Valid Param 
 Accurate Covar 
 PosDef 
 Made PosDef 
 
 
 False 
 True 
 False 
 False 
 False 
 
 
 Hesse Fail 
 HasCov 
 Above EDM 
 
 Reach calllim 
 
 
 True 
 True 
 False 
 
 False

+,Name,Value,Hesse Error,Minos Error-,Minos Error+,Limit-,Limit+,Fixed?
0,par_000_lon_0,1,0,,,,,No
1,par_001_lat_0,1,0,,,,,No
2,par_002_index,2.2,0,,,,,No
3,par_003_amplitude,3,0,,,,,No
4,par_004_reference,1,1,,,,,Yes
5,par_005_norm,1,0,,,,,No
6,par_006_tilt,0,1,,,,,Yes
7,par_007_reference,1,1,,,,,Yes
8,par_008_norm,1,0,,,,,No
9,par_009_tilt,0,1,,,,,Yes


RuntimeError: Covariance is not valid. May be the last Hesse call failed?

In [14]:
fit.datasets.likelihood(None)


0.0